In [ ]:
#
# Analysis tools for indirect vect with examples of how to use them.
#

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

exps = [
    (1,1),
    (1,32),
    (1,64),
    (1,128),
    (2,32),
    (2,64),
    (2,128),
    (4,32),
    (4,64),
    (4,128),
]

In [ ]:
!ls ../../candidate-solutions/gp_mutate_indirect_vect_1_128/0.0.1_20210204-200800

In [ ]:
plt.figure(figsize=(10,8))
for depth, width in exps:
    stats_path = glob.glob(f"../../candidate-solutions/gp_mutate_indirect_vect_{depth}_{width}/*/experiment_stats.csv")[0]
    df = pd.read_csv(stats_path)
    plt.plot(df.generation, df.max_score, label=f"{depth} {width}")
plt.ylabel("Fitness")
plt.xlabel("Generation")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
for depth, width in exps:
    stats_path = glob.glob(f"../../candidate-solutions/gp_mutate_indirect_vect_{depth}_{width}/*/experiment_stats.csv")[0]
    df = pd.read_csv(stats_path)
    plt.plot(df.generation[:200], df.max_score[:200], label=f"{depth} {width}")
plt.ylabel("Fitness")
plt.xlabel("Generation")
plt.legend()
plt.show()

In [ ]:
stats_path = glob.glob(f"../../candidate-solutions/gp_mutate_indirect_vect_1_128/*/experiment_stats.csv")[0]
df = pd.read_csv(stats_path)
df.generation

In [ ]:
from domain.indirect_vect.generate_vector import generate_integer_vector
from domain.leading_one.leading_one_fitness import leading_one_fitness
from copy import deepcopy
from keras.models import load_model


mutants_per_model = 100
gens = np.arange(1,177,1)
mean_diffs = []
mean_mutant_fitnesses = []
mean_lowest_flip = []
most_flipped = []
mean_flips_so_far = []
gens_so_far = []
for i, g in enumerate(gens):
    print("Processing gen", g)
    
    # Load best model that generation
    model_path = glob.glob(f'../../candidate-solutions/gp_mutate_indirect_vect_1_128/0.0.1_20210204-200800/{g}/*')[0]
    model = load_model(model_path)
    
    # Extract initial weights
    model_weights = {}
    for layer in model.layers:
        model_weights[layer.name] = layer.get_weights()  
    
    # Extract initial vector
    model_vector = generate_integer_vector(model)
    
    # Generate mutants
    diffs = []
    mutant_fitnesses = []
    lowest_flips = []
    total_flip = np.zeros(len(model_vector))
    mean_flips = []
    for m in range(mutants_per_model):
        
        # Generate mutant weights
        mutant_weights = deepcopy(model_weights)
        mean = 0
        standard_deviation = 0.1
        mutation_rate = 0.9
        for layer_name in model_weights:
            for i in range(len(model_weights[layer_name])):
                weights_shape = model_weights[layer_name][i].shape
                is_mutate = np.random.random(weights_shape) < mutation_rate
                gaussian_noise = np.random.normal(mean, standard_deviation, size=weights_shape)
                mutant_weights[layer_name][i][is_mutate] += gaussian_noise[is_mutate]   
                
        # Generate mutant vector
        for layer in model.layers:
            layer.set_weights(mutant_weights[layer.name])
        mutant_vector = generate_integer_vector(model)
        
        # Compute diff from mutation
        diff = np.sum(np.abs(model_vector - mutant_vector))
        mutant_fitness = leading_one_fitness(mutant_vector)
        total_flip += np.abs(model_vector - mutant_vector)
        
        if diff > 0:
            lowest_flip = np.argmax(np.abs(model_vector - mutant_vector))
            lowest_flips.append(lowest_flip)
            
            mean_flips.append(np.sum(np.arange(len(model_vector)) * np.abs(model_vector - mutant_vector)) / float(diff))
        
        mutant_fitnesses.append(mutant_fitness)
        diffs.append(diff)
    
    mean_diffs.append(np.mean(diffs))
    mean_flips_so_far.append(np.mean(mean_flips))
    mean_mutant_fitnesses.append(np.mean(mutant_fitnesses))
    mean_lowest_flip.append(np.mean(lowest_flip))
    most_flipped.append(np.argmax(total_flip))
    gens_so_far.append(g)

    plt.plot(gens_so_far, mean_diffs)
    plt.title('Mean Diffs')
    plt.show()
    

    plt.plot(gens_so_far, mean_mutant_fitnesses)
    plt.title('Mean mutant fitness')
    plt.show()
    
    plt.plot(gens_so_far, mean_lowest_flip)
    plt.title('Mean lowest flip')
    plt.show()
    
    plt.plot(gens_so_far, most_flipped)
    plt.title('Most flipped')
    plt.show()
    
    plt.plot(gens_so_far, mean_flips_so_far)
    plt.title('Mean Flip Location')
    plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))
ax1.set_xlabel('Generation')
ax1.set_ylabel('Best Candidate Fitness', color='blue')
ax1.plot(df.generation, df.max_score, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Mean bits flipped by mutation of Best Candidate', color='green')
ax2.plot(gens_so_far, mean_diffs, color='green')
ax2.tick_params(axis='y', labelcolor='green')

#plt.title('Best Candidate by Generation')

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))
ax1.set_xlabel('Generation')
ax1.set_ylabel('Best Candidate Fitness', color='blue')
ax1.plot(df.generation, df.max_score, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Most often flipped bit by mutation of Best Candidate', color='green')
ax2.plot(gens_so_far, most_flipped, color='green')
ax2.tick_params(axis='y', labelcolor='green')

#plt.title('Best Candidate by Generation')

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))
ax1.set_xlabel('Generation')
ax1.set_ylabel('Best Candidate Fitness', color='blue')
ax1.plot(df.generation, df.max_score, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Mean bit flipped when mutated', color='green')
ax2.plot(gens_so_far, mean_flips_so_far, color='green')
ax2.tick_params(axis='y', labelcolor='green')

#plt.title('Best Candidate by Generation')

plt.show()